# Model 2


Stochastic optimization model used to solve the two-stage problem

In [1]:
using JuMP,HiGHS,DataFrames
using CSV
using DataFrames

### Data Imports

In [2]:
dateiStart = "2_buses_"

# Import data about buses
df_buses = CSV.read(string("data/",dateiStart,"paperdata.csv"), DataFrame,types=Dict("C" => String,"I" => String, "C_cur" => String, "Lambda" => String))
#Import demand data
df_demands = CSV.read(string("data/",dateiStart,"demand.csv"), DataFrame)
# Import the realized wind energy
df_realizedEnergy = CSV.read(string("data/",dateiStart,"realizedEnergy.csv"), DataFrame)
# Import the remaining data
df_other = CSV.read(string("data/",dateiStart,"other_data.csv"), DataFrame)

Row,Omega,Pi
,Int64,Float64
1,1,0.6
2,2,0.4


In [3]:
"""
Transforms indices into range for lists of objects (such as generation units)
Input of 0 or empty cells is translated to no objects (i.e. empty array)
"""
function string_to_range(s)
    # Check if s is missing or a string that represents an empty cell
    if s == "missing" || s == "0"
        return Int[]  # Return an empty array of Int type
    else
        str = string(s)  # Convert to string if not already
        nums = split(str, ";")  # Split the string by comma
        nums = parse.(Int, nums)  # Parse each part to an integer
        return minimum(nums):maximum(nums)  # Return the range
    end
end

"""
Transforms indices into array of values (such as generation costs per unit for each generation unit)
Input of missing cells is translated to no values (i.e. empty array)
Input of 0 is translated to 0 since it can be a coefficient
"""
function string_to_array(s)
    # Check if s is missing or a string that represents an empty cell
    #if s == "0" return 0  # Return an empty array of Float64 type

    if s == "missing"
        return Float64[]
    else
        str = string(s)  # Convert to string if not already
        nums = split(str, ";")  # Split the string by semicolon
        nums = parse.(Float64, nums)  # Parse each part to a float
        return nums  # Return the array of floats
    end
end
"""
Transforms a range and an array into one dictionary
"""
function array_to_dict(keys::AbstractVector{T}, values::AbstractVector{S}) where {T,S}
    # Überprüfen, ob die Länge der Arrays gleich ist
    if length(keys) != length(values)
        throw(ArgumentError("Die Länge der Arrays muss gleich sein"))
    end

    # Initialisierung des leeren Dictionary
    result_dict = Dict{T,S}()

    # Hinzufügen von Schlüssel-Wert-Paaren zum Dictionary
    for i in 1:length(keys)
        result_dict[keys[i]] = values[i]
    end

    return result_dict
end


array_to_dict

In [4]:
# Create an empty dictionary to hold all nodes
data = Dict()

# Iterate over the range of nodes
for singleNode in eachrow(df_buses)
    # Create a dictionary for the current node with all other variables
    data_node = Dict(
        :I => string_to_array(string.(singleNode.I)),
        :Q => string_to_array(string.(singleNode.Q)),
        :J => string_to_array(string.(singleNode.J)),
        :Λ => string_to_array(string.(singleNode.Lambda)),
        :C_save => string_to_array(string(singleNode.C)) ,
        :C_RU_save => string_to_array(string(singleNode.C_RU)),
        :C_RD_save => string_to_array(string(singleNode.C_RD)),
        :C_U_save => string_to_array(string(singleNode.C_U)),
        :C_D_save => string_to_array(string(singleNode.C_D)),
        :V_LOL_save => string_to_array(string(singleNode.V_LOL)),
        :C_cur_save => string_to_array(string(singleNode.C_cur)),
        :P_max_save => string_to_array(string(singleNode.P_max)),
        :b_save => string_to_array(string(singleNode.b)),
        :LC_Max_save => string_to_array(string(singleNode.LC_Max)),
        :L_save => Dict(),
        :C => Dict(),
        :C_RU => Dict(),
        :C_RD => Dict(),
        :C_U => Dict(),
        :C_D => Dict(),
        :P_max => Dict(),
        :C_cur => Dict(),
        :b => Dict(),
        :LC_Max => Dict(),
        :V_LOL => Dict(),
        :Ω => 0,
        :T => 1:1,
    )    
    # Assign the dictionary to the current node key
    data[singleNode.Node] = data_node
end

println(data[2][:Q])



Float64[]


In [5]:
for singleNode in keys(data)
    # Create a dictionary for the current node with all other variables
    data[singleNode][:C] = array_to_dict(data[singleNode][:I], data[singleNode][:C_save])
    data[singleNode][:C_RU] = array_to_dict(data[singleNode][:I], data[singleNode][:C_RU_save])
    data[singleNode][:C_RD] = array_to_dict(data[singleNode][:I], data[singleNode][:C_RD_save])
    data[singleNode][:C_U] = array_to_dict(data[singleNode][:I], data[singleNode][:C_U_save])
    data[singleNode][:C_D] = array_to_dict(data[singleNode][:I], data[singleNode][:C_D_save])
    data[singleNode][:P_max] = array_to_dict(data[singleNode][:I], data[singleNode][:P_max_save])
    data[singleNode][:C_cur] = array_to_dict(data[singleNode][:Q], data[singleNode][:C_cur_save])
    data[singleNode][:b] = array_to_dict(data[singleNode][:Λ], data[singleNode][:b_save])
    data[singleNode][:LC_Max] = array_to_dict(data[singleNode][:Λ], data[singleNode][:LC_Max_save])
    data[singleNode][:V_LOL] = array_to_dict(data[singleNode][:J], data[singleNode][:V_LOL_save])
end

In [6]:
#Create dictionary for alle demands dependent on node and load -> array with different times
knownLoads = []
demands = Dict()
defaultPeriods = 1
for singleNode in eachrow(df_demands)
    data_node = Dict(
        :L => string_to_array(string(singleNode.Demands)),
    )    
    if  singleNode.Node ∉ knownLoads
        demands = Dict()
    end
    push!(knownLoads, singleNode.Node)

    demands[singleNode.Loads] = data_node
    data[singleNode.Node][:L] = demands
    
    data[singleNode.Node][:T] = 1 : singleNode.Periods
    if defaultPeriods < singleNode.Periods
        defaultPeriods = singleNode.Periods
    end
end

#Fill periods of Nodes without demands -> Set Default Value
for singleNode in keys(data)
    #if data[singleNode][:T] === nothing
        data[singleNode][:T] = minimum(1):maximum(defaultPeriods)
    #end
end


In [7]:
#create dictionary with all scenarios
scenarios = Dict()
for singleNode in eachrow(df_other)
    data_node = Dict(
        :π => singleNode.Pi,
    )
    scenarios[singleNode.Omega] = data_node
end

#create dictionary with all realized Energy from wind turbine dependent on node, wind turbine, scenario -> array with different times
windenergy = Dict()
data_Omega = Dict()
for singleNode in eachrow(df_realizedEnergy)
    data_node = Dict(
        :W_realized => string_to_array(string(singleNode.energy)),
    )
    if ! haskey(windenergy, singleNode.Q)
        data_Omega = Dict()
    end
    data_Omega[singleNode.Omega] = data_node
    windenergy[singleNode.Q] = data_Omega
    data[singleNode.Node][:Windenergy] = windenergy
end


In [8]:
#Float muss in Integer umgewandelt werden
for value in keys(data)
    data[value][:Λ]  = round.(Int, data[value][:Λ])
end

### Functions for Variables and Constraints

In [9]:
# Initializes variables
function init_variables(model::JuMP.Model)
    P=@variable(model, P[n in keys(data), i in data[n][:I], t in data[n][:T]] >=0) #energy generated
    R_U=@variable(model, R_U[n in keys(data), i in data[n][:I], t in data[n][:T]] >=0) #committed upward reserve capacity of generator i
    R_D=@variable(model, R_D[n in keys(data), i in data[n][:I], t in data[n][:T]] >=0) #committed downward reserve capacity of generator i
    r_U=@variable(model, r_U[n in keys(data), i in data[n][:I], ω in keys(scenarios), t in data[n][:T]] >=0) #up regulation of generator i in case 𝜔
    r_D=@variable(model, r_D[n in keys(data), i in data[n][:I], ω in keys(scenarios), t in data[n][:T]] >=0) #down regulation of generator i in case 𝜔
    L_Shed=@variable(model, L_Shed[n in keys(data), j in data[n][:J], ω in keys(scenarios), t in data[n][:T]] >=0) #loss of load at load demand j in case 𝜔
    W_spill=@variable(model, W_spill[n in keys(data), q in data[n][:Q], ω in keys(scenarios), t in data[n][:T]] >=0) #curtailment of turbine q in case 𝜔
    W_s=@variable(model, W_s[n in keys(data), q in data[n][:Q], t in data[n][:T]] >=0) #scheduled wind power generation at turbine q
    𝛿=@variable(model, 𝛿[n in keys(data), ℓ in data[n][:Λ], t in data[n][:T]] >=0) #voltage angle
    𝛿_scenario=@variable(model, 𝛿_scenario[n in keys(data), ℓ in data[n][:Λ], ω in keys(scenarios),t in data[n][:T]] >=0) #voltage angle in case 𝜔
    PF=@variable(model, PF[n in keys(data), ℓ in data[n][:Λ], t in data[n][:T]]) #power flow
    PF_scenario=@variable(model, PF_scenario[n in keys(data), ℓ in data[n][:Λ], ω in keys(scenarios), t in data[n][:T]]) #power flow in case 𝜔
    
    
    vars= Dict(
        :P => P,
        :R_U => R_U,
        :R_D => R_D,
        :r_U => r_U,
        :r_D => r_D,
        :L_Shed => L_Shed,
        :W_spill => W_spill,
        :W_s => W_s,
        :𝛿 => 𝛿,
        :𝛿_scenario => 𝛿_scenario,
        :PF => PF,
        :PF_scenario => PF_scenario,
    )    
    return vars
end;

In [10]:
# Initializes constraints
function init_constraints(model::JuMP.Model, data::Dict, vars::Dict)
    @constraints(model, begin 
    c1[n in keys(data),i in data[n][:I], t in data[n][:T]], vars[:P][n, i, t]+ vars[:R_U][n, i, t]<= data[n][:P_max][i]
    
    c2[n in keys(data),i in data[n][:I] ,t in data[n][:T]], vars[:P][n, i, t]- vars[:R_D][n, i, t]>= 0
    c3[n in keys(data),i in data[n][:I], t in data[n][:T], ω in keys(scenarios)], vars[:r_U][n, i, ω, t] <= vars[:R_U][n,i,t]
    c4[n in keys(data),i in data[n][:I], t in data[n][:T], ω in keys(scenarios)], vars[:r_D][n, i, ω, t] <= vars[:R_D][n,i,t]

    c5[n in keys(data), ℓ in data[n][:Λ], t in data[n][:T]], vars[:PF][n, ℓ, t] <= data[n][:LC_Max][ℓ]
    c5_scenario[ω in keys(scenarios), n in keys(data), ℓ in data[n][:Λ], t in data[n][:T]], vars[:PF_scenario][n, ℓ, ω, t] <= data[n][:LC_Max][ℓ]
    
    c6[n in keys(data), ℓ in data[n][:Λ], t in data[n][:T]], (vars[:𝛿][n, ℓ, t] - vars[:𝛿][ℓ, n, t]) * 1 / data[n][:b][ℓ]  == vars[:PF][n, ℓ, t]
    c6_scenario[ω in keys(scenarios), n in keys(data), ℓ in data[n][:Λ], t in data[n][:T]], (vars[:𝛿_scenario][n, ℓ, ω, t] - vars[:𝛿_scenario][ℓ, n, ω, t]) * 1 / data[n][:b][ℓ] == vars[:PF_scenario][n, ℓ, ω, t]
       
    #steht im Lehrbuch, aber macht keinen Sinn -> muss auskommentiert bleiben
    #c8[n in keys(data), t in data[n][:T]], vars[:𝛿][n, data[n][:Λ][1], t] == 0
    #c81[ω in keys(scenarios), n in keys(data), t in data[n][:T]], vars[:𝛿_scenario][ω, n, data[n][:Λ][1], t] == 0
    #c11, vars[:𝛿][1,2,1]==0  ### hard coded
    #c12[ω in keys(scenarios)], vars[:𝛿_scenario][1,2,ω,1]==0 ### hard coded
    
    c9[ω in keys(scenarios), n in keys(data),t in data[n][:T], q in data[n][:Q]], vars[:W_spill][n,q,ω,t] <= data[n][:Windenergy][q][ω][:W_realized][t]
    c10[ω in keys(scenarios), n in keys(data),t in data[n][:T], j in data[n][:J]], vars[:L_Shed][n,j, ω, t] <= data[n][:L][j][:L][t]

    Power_balance_day_ahead[n in keys(data), t in data[n][:T]], 
    sum(vars[:P][n,i,t] for i in data[n][:I]) + sum(vars[:W_s][n,q,t] for q in data[n][:Q]) - sum(data[n][:L][j][:L][t] for j in data[n][:J]) - sum(vars[:PF][n, ℓ, t] for ℓ in data[n][:Λ]) == 0

    Power_balance_at_stage[ω in keys(scenarios), n in keys(data), t in data[n][:T]], 
    sum(vars[:r_U][n,i, ω, t] for i in data[n][:I]) -
    sum(vars[:r_D][n,i, ω, t] for i in data[n][:I]) +
    sum(vars[:L_Shed][n,j, ω, t] for j in data[n][:J]) +
    sum(data[n][:Windenergy][q][ω][:W_realized][t] -vars[:W_s][n,q,t] - vars[:W_spill][n,q,ω,t] for q in data[n][:Q]) -
    sum(vars[:PF_scenario][n, ℓ, ω, t] for ℓ in data[n][:Λ]) +
    sum(vars[:PF][n, ℓ, t] for ℓ in data[n][:Λ]) == 0
    end)
end;

In [11]:
# Initializes objective function
function init_obj_function(model, data, vars)
    @objective(model, Min, sum(vars[:P][n,i,t]* data[n][:C][i] + vars[:R_U][n,i,t] * data[n][:C_RU][i] + vars[:R_D][n,i,t] * data[n][:C_RD][i] for n in keys(data), i in data[n][:I], t in data[n][:T]) +
    sum(scenarios[ω][:π] * (sum(data[n][:C_U][i] * vars[:r_U][n,i, ω ,t] - data[n][:C_D][i] * vars[:r_D][n,i, ω, t] for i in data[n][:I]) +
                sum(data[n][:C_cur][q] * vars[:W_spill][n,q, ω, t] for q in data[n][:Q]) +
                sum(data[n][:V_LOL][j] * vars[:L_Shed][n, j, ω, t] for j in data[n][:J])) for n in keys(data), ω in keys(scenarios), t in data[n][:T])
    )
end;

### Model Creation

In [15]:
# Create and print model
model=Model(HiGHS.Optimizer)
vars=init_variables(model)
init_constraints(model, data, vars)
init_obj_function(model, data, vars)
print(model)

Min 35 P[2,3.0,1] + 10 R_U[2,3.0,1] + 9 R_D[2,3.0,1] + 10 P[1,1.0,1] + 16 R_U[1,1.0,1] + 15 R_D[1,1.0,1] + 30 P[1,2.0,1] + 13 R_U[1,2.0,1] + 12 R_D[1,2.0,1] + 14 r_U[2,3.0,2,1] - 14 r_D[2,3.0,2,1] + 80 L_Shed[2,2.0,2,1] + 21 r_U[2,3.0,1,1] - 21 r_D[2,3.0,1,1] + 120 L_Shed[2,2.0,1,1] + 4 r_U[1,1.0,2,1] - 4 r_D[1,1.0,2,1] + 12 r_U[1,2.0,2,1] - 12 r_D[1,2.0,2,1] + 80 L_Shed[1,1.0,2,1] + 6 r_U[1,1.0,1,1] - 6 r_D[1,1.0,1,1] + 18 r_U[1,2.0,1,1] - 18 r_D[1,2.0,1,1] + 120 L_Shed[1,1.0,1,1]
Subject to


 c6[2,1,1] : 7.692307692307692 𝛿[2,1,1] - 7.692307692307692 𝛿[1,2,1] - PF[2,1,1] == 0
 c6[2,2,1] : -PF[2,2,1] == 0
 c6[1,1,1] : -PF[1,1,1] == 0
 c6[1,2,1] : -7.692307692307692 𝛿[2,1,1] + 7.692307692307692 𝛿[1,2,1] - PF[1,2,1] == 0
 c6_scenario[2,2,1,1] : 7.692307692307692 𝛿_scenario[2,1,2,1] - 7.692307692307692 𝛿_scenario[1,2,2,1] - PF_scenario[2,1,2,1] == 0
 c6_scenario[2,2,2,1] : -PF_scenario[2,2,2,1] == 0
 c6_scenario[2,1,1,1] : -PF_scenario[1,1,2,1] == 0
 c6_scenario[2,1,2,1] : -7.692307692307692 𝛿_scenario[2,1,2,1] + 7.692307692307692 𝛿_scenario[1,2,2,1] - PF_scenario[1,2,2,1] == 0
 c6_scenario[1,2,1,1] : 7.692307692307692 𝛿_scenario[2,1,1,1] - 7.692307692307692 𝛿_scenario[1,2,1,1] - PF_scenario[2,1,1,1] == 0
 c6_scenario[1,2,2,1] : -PF_scenario[2,2,1,1] == 0
 c6_scenario[1,1,1,1] : -PF_scenario[1,1,1,1] == 0
 c6_scenario[1,1,2,1] : -7.692307692307692 𝛿_scenario[2,1,1,1] + 7.692307692307692 𝛿_scenario[1,2,1,1] - PF_scenario[1,2,1,1] == 0
 Power_balance_day_ahead[2,1] : P[2,3.0,1]

In [13]:
optimize!(model)

Running HiGHS 1.6.0: Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
29 rows, 39 cols, 86 nonzeros
22 rows, 29 cols, 68 nonzeros
22 rows, 29 cols, 68 nonzeros
Presolve : Reductions: rows 22(-32); columns 29(-23); elements 68(-54)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0    -8.4999773688e+01 Ph1: 11(15); Du: 7(84.9998) 0s
         18     2.6200000000e+03 Pr: 0(0) 0s
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Simplex   iterations: 18
Objective value     :  2.6200000000e+03
HiGHS run time      :          0.00


In [14]:
# Print the values of the variables
println("Variable values:")
println("P:")
for n in keys(data), i in data[n][:I], t in data[n][:T]
    println("P[$n, $i, $t]: ", JuMP.value.(vars[:P][n, i, t]))
end

println("\nR_U:")
for n in keys(data), i in data[n][:I], t in data[n][:T]
    println("R_U[$n, $i, $t]: ", value(vars[:R_U][n,i, t]))
end

println("\nR_D:")
for n in keys(data), i in data[n][:I], t in data[n][:T]
    println("R_D[$n, $i, $t]: ", value(vars[:R_D][n,i, t]))end

println("\nL_Shed:")
for n in keys(data), j in data[n][:J], ω in keys(scenarios), t in data[n][:T]
    println("L_Shed[$n, $j, $ω, $t]: ", value(vars[:L_Shed][n,j, ω, t]))
end

println("\nW_spill:")
for n in keys(data), q in data[n][:Q], ω in keys(scenarios), t in data[n][:T]
    println("W_spill[$n, $q, $ω, $t]: ", value(vars[:W_spill][n,q, ω, t]))
end

println("\nW_s:")
for n in keys(data), q in data[n][:Q], t in data[n][:T]
    println("W_s[$n, $q, $t]: ", value(vars[:W_s][n,q, t]))
end

println("\nPF[n, ℓ, t]:")
for n in keys(data), ℓ in data[n][:Λ], t in data[n][:T]
    println("PF[$n, $ℓ, $t]: ", value(vars[:PF][n, ℓ, t]))
end

println("\nPF_scenario[ω, n, ℓ, t]:")
for ω in keys(scenarios), n in keys(data), ℓ in data[n][:Λ], t in data[n][:T]
    println("PF_scenario[$ω, $n, $ℓ, $t]: ", value(vars[:PF_scenario][ω, n, ℓ, t]))
end

println("\nr_U[n,i, ω, t]:")
for n in keys(data), i in data[n][:I], ω in keys(scenarios), t in data[n][:T]
    println("r_U[$n, $i, $ω, $t]: ", value(vars[:r_U][n,i, ω, t]))
end

println("\nr_D[n,i, ω, t]:")
for n in keys(data), i in data[n][:I], ω in keys(scenarios), t in data[n][:T]
    println("r_D[$n, $i, $ω, $t]: ", value(vars[:r_D][n,i, ω, t]))
end

println("\nData values:")

println("\nW_realized:")
for n in keys(data), q in data[n][:Q], ω in keys(scenarios), t in data[n][:T]
    println("W_realized[$n, $q, $ω, $t]: ", value(data[n][:Windenergy][q][ω][:W_realized][t]))
end

println("\nLoad:")
for n in keys(data), j in data[n][:J], t in data[n][:T]
    println("L[$n, $j, $t]:", value(data[n][:L][j][:L][t]))
end

Variable values:
P:


P[2, 3.0, 1]: 40.0
P[1, 1.0, 1]: 50.0
P[1, 2.0, 1]: 40.0

R_U:
R_U[2, 3.0, 1]: 0.0
R_U[1, 1.0, 1]: 0.0
R_U[1, 2.0, 1]: 0.0

R_D:
R_D[2, 3.0, 1]: 40.0
R_D[1, 1.0, 1]: 0.0
R_D[1, 2.0, 1]: -0.0

L_Shed:


L_Shed[2, 2.0, 2, 1]: 0.0
L_Shed[2, 2.0, 1, 1]: 0.0
L_Shed[1, 1.0, 2, 1]: 0.0
L_Shed[1, 1.0, 1, 1]: 0.0

W_spill:
W_spill[1, 1.0, 2, 1]: 0.0
W_spill[1, 1.0, 1, 1]: 0.0

W_s:
W_s[1, 1.0, 1]: 10.0

PF[n, ℓ, t]:
PF[2, 1, 1]: -60.0
PF[2, 2, 1]: -0.0
PF[1, 1, 1]: -0.0
PF[1, 2, 1]: 60.0

PF_scenario[ω, n, ℓ, t]:
PF_scenario[2, 2, 1, 1]: -0.0
PF_scenario[2, 2, 2, 1]: -0.0
PF_scenario[2, 1, 1, 1]: -100.0
PF_scenario[2, 1, 2, 1]: -60.0
PF_scenario[1, 2, 1, 1]: 100.0
PF_scenario[1, 2, 2, 1]: 60.0
PF_scenario[1, 1, 1, 1]: -0.0
PF_scenario[1, 1, 2, 1]: -0.0



r_U[n,i, ω, t]:
r_U[2, 3.0, 2, 1]: -0.0
r_U[2, 3.0, 1, 1]: 0.0
r_U[1, 1.0, 2, 1]: -0.0
r_U[1, 1.0, 1, 1]: -0.0
r_U[1, 2.0, 2, 1]: -0.0
r_U[1, 2.0, 1, 1]: 0.0

r_D[n,i, ω, t]:
r_D[2, 3.0, 2, 1]: 0.0
r_D[2, 3.0, 1, 1]: 40.0
r_D[1, 1.0, 2, 1]: 0.0
r_D[1, 1.0, 1, 1]: 0.0
r_D[1, 2.0, 2, 1]: 0.0
r_D[1, 2.0, 1, 1]: 0.0

Data values:

W_realized:
W_realized[1, 1.0, 2, 1]: 10.0
W_realized[1, 1.0, 1, 1]: 50.0

Load:


L[2, 2.0, 1]:100.0
L[1, 1.0, 1]:40.0
